In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers


In [10]:
# User Inputs
dataset_choice = 'mnist'      # 'mnist' or 'fashion'
epochs = 50
batch_size = 128
noise_dim = 100
learning_rate = 0.0002
save_interval = 5


In [11]:
if dataset_choice == 'mnist':
    (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
elif dataset_choice == 'fashion':
    (x_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
else:
    raise ValueError("Invalid dataset choice")

# Normalize to [-1, 1]
x_train = (x_train.astype('float32') - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

img_shape = x_train.shape[1:]


In [12]:
os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)


In [13]:
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=noise_dim),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(momentum=0.8),

        layers.Dense(512),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(momentum=0.8),

        layers.Dense(1024),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(momentum=0.8),

        layers.Dense(np.prod(img_shape), activation='tanh'),
        layers.Reshape(img_shape)
    ])
    return model


In [14]:
def build_discriminator():
    model = models.Sequential([
        layers.Flatten(input_shape=img_shape),
        layers.Dense(512),
        layers.LeakyReLU(0.2),

        layers.Dense(256),
        layers.LeakyReLU(0.2),

        layers.Dense(1, activation='sigmoid')
    ])
    return model

discriminator = build_discriminator()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5),
    metrics=['accuracy']
)


In [16]:
discriminator.trainable = False

generator = build_generator()
gan_input = layers.Input(shape=(noise_dim,))
fake_img = generator(gan_input)
gan_output = discriminator(fake_img)

gan = models.Model(gan_input, gan_output)

gan.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(learning_rate, 0.5)
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
def save_images(epoch):
    noise = np.random.normal(0, 1, (25, noise_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(5, 5, figsize=(5, 5))
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(gen_imgs[idx, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            idx += 1

    plt.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()


In [18]:
half_batch = batch_size // 2

for epoch in range(1, epochs + 1):

    # Train Discriminator
    idx = np.random.randint(0, x_train.shape[0], half_batch)
    real_imgs = x_train[idx]

    noise = np.random.normal(0, 1, (half_batch, noise_dim))
    fake_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    print(
        f"Epoch {epoch}/{epochs} | "
        f"D_loss: {d_loss[0]:.2f} | "
        f"D_acc: {d_loss[1]*100:.2f}% | "
        f"G_loss: {g_loss:.2f}"
    )

    if epoch % save_interval == 0:
        save_images(epoch)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step 


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50 | D_loss: 1.25 | D_acc: 15.62% | G_loss: 0.85
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Epoch 2/50 | D_loss: 1.13 | D_acc: 24.87% | G_loss: 0.83
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 3/50 | D_loss: 1.10 | D_acc: 25.39% | G_loss: 0.80
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Epoch 4/50 | D_loss: 1.10 | D_acc: 23.93% | G_loss: 0.78
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 5/50 | D_loss: 1.11 | D_acc: 22.18% | G_loss: 0.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 6/50 | D_loss: 1.11 | D_acc: 20.32% | G_loss: 0.73
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Epoch 7/50 | D_loss: 1.12 | D_acc: 18.24% | G_loss: 0.70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 8/50 | D_loss: 1.13 | D_acc: 16.49% | G_loss: 0.68
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Epoch 9/50 | D_loss: 1.14 | D_acc: 15.01% | G_loss: 0.66
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Epoch 10/50 | D_loss: 1.15 | D_acc: 13.79% | G_loss: 0.64
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2/2 ━━━━━━

In [19]:
noise = np.random.normal(0, 1, (100, noise_dim))
final_images = generator.predict(noise)
final_images = (final_images + 1) / 2.0

for i in range(100):
    plt.imsave(
        f"final_generated_images/img_{i+1}.png",
        final_images[i, :, :, 0],
        cmap='gray'
    )


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


In [20]:
classifier = tf.keras.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, (3,3)),
    layers.LeakyReLU(0.2),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3)),
    layers.LeakyReLU(0.2),
    layers.MaxPooling2D(),

    layers.Flatten(),

    layers.Dense(128),
    layers.LeakyReLU(0.2),

    layers.Dense(10, activation='softmax')
])

classifier.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [21]:
# Reload labels properly
if dataset_choice == 'mnist':
    (_, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
else:
    (_, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

classifier.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    verbose=1
)


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8799 - loss: 0.3965
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9852 - loss: 0.0469
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9897 - loss: 0.0345
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9930 - loss: 0.0221
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9948 - loss: 0.0170


In [22]:
generated_imgs = final_images.reshape(-1, 28, 28, 1)

predictions = classifier.predict(generated_imgs)
predicted_labels = np.argmax(predictions, axis=1)

unique, counts = np.unique(predicted_labels, return_counts=True)

print("Label Distribution of Generated Images:")
for u, c in zip(unique, counts):
    print(f"Label {u}: {c} images")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step
Label Distribution of Generated Images:
Label 8: 100 images
